In [1]:
import math
import networkx as nx

# Define the search algorithms
def depth_first_search(start, goal, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    if start == goal:
        return [start]
    for next_city in city_distances[start]:
        if next_city not in visited:
            path = depth_first_search(next_city, goal, visited)
            if path:
                return [start] + path
    return None

def breadth_first_search(start, goal):
    queue = [(start, [start])]
    while queue:
        (current_city, path) = queue.pop(0)
        if current_city == goal:
            return path
        for next_city in city_distances[current_city]:
            if next_city not in path:
                queue.append((next_city, path + [next_city]))
    return None

def a_star_search(start, goal):
    queue = [(heuristic_distances[start], 0, start, [])]
    visited = set()
    while queue:
        _, distance, current_city, path = queue.pop(0)
        if current_city == goal:
            return path + [current_city], distance
        visited.add(current_city)
        for next_city in city_distances[current_city]:
            if next_city not in visited:
                new_distance = distance + city_distances[current_city][next_city]
                heuristic_distance = heuristic_distances[next_city]
                queue.append((new_distance + heuristic_distance, new_distance, next_city, path + [current_city]))
        queue.sort()
    return None

def main():
  # Define the map
  # Define the heuristic distances (straight-line distances)
  heuristic_distances = {
      "Manchester": 0,
      "Holyhead": 0,
      "Liverpool":0 ,
      "York": 0,
      "Carlisle": 0,
      "Newcastle": 0,
      "Glasgow": 0,
      "Edinburgh": 0,
      "Oban": 0,
      "Aberdeen": 0,
      "Inverness": 0
  }
  city_distances = {
      "Manchester": {"Liverpool": 54, "York": 113, "Newcastle": 235, "Edinburgh": 354, "Carlisle": 196},
      "Holyhead": {"Liverpool": 150},
      "Liverpool": {"Manchester": 55, "Holyhead": 150},
      "York": {"Manchester": 112, "Newcastle": 136},
      "Carlisle": {"Manchester": 195, "Glasgow":154},
      "Newcastle": {"Manchester": 231,"York": 136, "Edinburgh": 167},
      "Glasgow": {"Edinburgh": 75,"Carlisle": 154,"Oban": 155,"Inverness": 273,"Aberdeen": 233,},
      "Edinburgh": {"Manchester": 344,"Newcastle": 166,"Glasgow": 74,},
      "Oban": {"Glasgow": 154, "Inverness": 172},
      "Aberdeen": {"Inverness": 167,"Glasgow": 230},
      "Inverness": {"Glasgow": 271, "Oban": 174, "Aberdeen": 167}
  }

  cities =["Manchester","Liverpool","Holyhead","York","Newcastle","Edinburgh","Glasgow","Oban","Inverness","Aberdeen","Carlisle"]
  print("These are the cities available")
  for i in range(len(cities)):
    print(cities[i])
    
  
  start = None
  while start not in cities:
    try:
      start = input("please enter a place you want to go to: ")
    except:
      print("Please enter one of the cities provided(please capitalise the first letter)")
  goal = None
  while goal not in cities:
    try:
      goal = input("Please enter your end destination")

main()




These are the cities available
Manchester
Liverpool
Holyhead
York
Newcastle
Edinburgh
Glasgow
Oban
Inverness
Aberdeen
Carlisle
please enter a place you want to go to: Manchester
